In [1]:
import openai
import os
import json

from qdrant_client import QdrantClient
from qdrant_client.models import Filter, FieldCondition, MatchValue

from langsmith import Client

In [2]:
qdrant_client = QdrantClient(url="http://localhost:6333")

### Download data from DB

In [7]:
all_points = qdrant_client.scroll(
    collection_name="Amazon-items-collection-00",
    limit=100,
    offset=None,
    with_payload=True,
    with_vectors=False
)

In [8]:
all_points[0][0].payload

{'description': 'KEEPRO Pencil 2nd Generation for iPad, Magnetic Wireless Charge Tilt Sensitivity Palm Rejection Active Pen for Apple iPad Pro 11" 4/3/2/1, iPad Pro 12.9" 6/5/4/3, iPad Air 4/5, iPad Mini 6[Compatibility]- ONLY compatible with iPad mini (6th generation), iPad Air (4th and 5th generation), iPad Pro 12.9-inch (3rd, 4th, 5th and 6th generation), iPad Pro 11-inch (1st, 2nd, 3rd and 4th generation), check and confirm your device before place the order (Note: If the pen doesn\'t charge, fully charge your iPad first then try charging the pen again) [Charging and Pairs Magnetically]- Charges wirelessly, attaches and pairs magnetically to the compatible iPad, this pen is a preferable alternative to the Apple Pencil 2nd Generation [Tilt Sensitivity & Pixel Precision]- Pixel-perfect precision and industry-leading low latency with tilt sensitivity making drawing, sketching, coloring, taking notes, and marking up PDFs, as easy and natural as a real pencil [Native Palm Rejection]- Re

In [9]:
all_context = [{"id": data.payload["parent_asin"], "text": data.payload["description"]} for data in all_points[0]]

In [10]:
all_context

[{'id': 'B0BF18F6R7',
  'text': 'KEEPRO Pencil 2nd Generation for iPad, Magnetic Wireless Charge Tilt Sensitivity Palm Rejection Active Pen for Apple iPad Pro 11" 4/3/2/1, iPad Pro 12.9" 6/5/4/3, iPad Air 4/5, iPad Mini 6[Compatibility]- ONLY compatible with iPad mini (6th generation), iPad Air (4th and 5th generation), iPad Pro 12.9-inch (3rd, 4th, 5th and 6th generation), iPad Pro 11-inch (1st, 2nd, 3rd and 4th generation), check and confirm your device before place the order (Note: If the pen doesn\'t charge, fully charge your iPad first then try charging the pen again) [Charging and Pairs Magnetically]- Charges wirelessly, attaches and pairs magnetically to the compatible iPad, this pen is a preferable alternative to the Apple Pencil 2nd Generation [Tilt Sensitivity & Pixel Precision]- Pixel-perfect precision and industry-leading low latency with tilt sensitivity making drawing, sketching, coloring, taking notes, and marking up PDFs, as easy and natural as a real pencil [Native Pal

### Render a prompt to generate synthetic Eval reference dataset

In [11]:
output_schema = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "Suggested question.",
            },
            "chunk_ids": {
                "type": "array",
                "items": {
                    "type": "string",
                    "description": "ID of the chunk that could be used to answer the question.",
                },
            },
            "answer_example": {
                "type": "string",
                "description": "Suggested answer grounded in the context.",
            },
            "reasoning": {
                "type": "string",
                "description": "Reasoning why the question could be answered with the chunks.",
            },
        },
    },
}


SYSTEM_PROMPT = f"""
I am building a RAG application. I have a collection of 50 chunks of text.
The RAG application will act as a shopping assistant that can answer questions about the stock of the products we have available.
I will provide all of the available products to you with IDs of each chunk.
I want you to come up with 30 questions to which the answers could be grounded in the chunk context.
The questions should imitate a potential real user of this RAG system.
As an output I need you to provide me the list of questions and the IDs of the chunks that could be used to answer them.
Also, provide an example answer to the question given the context of the chunks.
Also, provide the reason why you chose the chunks to answer the questions.
Construct 10 questions that could use multipple chunks in the answer.
Construct 15 questions that could use single chunk in the answer.
Construct 5 questions that can't be answered with the available chunks.

<OUTPUT JSON SCHEMA>
{json.dumps(output_schema, indent=2)}
</OUTPUT JSON SCHEMA>

I need to be able to parse the json output.
"""

USER_PROMPT = f"""
Here is the list of chunks, each list element is a dictionary with id and text:
{all_context}
"""

In [12]:
print(SYSTEM_PROMPT)


I am building a RAG application. I have a collection of 50 chunks of text.
The RAG application will act as a shopping assistant that can answer questions about the stock of the products we have available.
I will provide all of the available products to you with IDs of each chunk.
I want you to come up with 30 questions to which the answers could be grounded in the chunk context.
The questions should imitate a potential real user of this RAG system.
As an output I need you to provide me the list of questions and the IDs of the chunks that could be used to answer them.
Also, provide an example answer to the question given the context of the chunks.
Also, provide the reason why you chose the chunks to answer the questions.
Construct 10 questions that could use multipple chunks in the answer.
Construct 15 questions that could use single chunk in the answer.
Construct 5 questions that can't be answered with the available chunks.

<OUTPUT JSON SCHEMA>
{
  "type": "array",
  "items": {
    "

In [13]:
print(USER_PROMPT)


Here is the list of chunks, each list element is a dictionary with id and text:
[{'id': 'B0BF18F6R7', 'text': 'KEEPRO Pencil 2nd Generation for iPad, Magnetic Wireless Charge Tilt Sensitivity Palm Rejection Active Pen for Apple iPad Pro 11" 4/3/2/1, iPad Pro 12.9" 6/5/4/3, iPad Air 4/5, iPad Mini 6[Compatibility]- ONLY compatible with iPad mini (6th generation), iPad Air (4th and 5th generation), iPad Pro 12.9-inch (3rd, 4th, 5th and 6th generation), iPad Pro 11-inch (1st, 2nd, 3rd and 4th generation), check and confirm your device before place the order (Note: If the pen doesn\'t charge, fully charge your iPad first then try charging the pen again) [Charging and Pairs Magnetically]- Charges wirelessly, attaches and pairs magnetically to the compatible iPad, this pen is a preferable alternative to the Apple Pencil 2nd Generation [Tilt Sensitivity & Pixel Precision]- Pixel-perfect precision and industry-leading low latency with tilt sensitivity making drawing, sketching, coloring, taki

In [14]:
response = openai.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT}
    ],
    reasoning_effort="minimal"
)

print(response.choices[0].message.content)

[
  {
    "question": "Do you have a stylus compatible with iPad Air 5th generation or iPad mini 6?",
    "chunk_ids": [
      "B0BF18F6R7"
    ],
    "answer_example": "Yes. The KEEPRO Pencil 2nd Generation (B0BF18F6R7) is listed as compatible with iPad Air (4th and 5th generation) and iPad mini (6th generation).",
    "reasoning": "The product description explicitly lists compatible iPad models including iPad Air 5th gen and iPad mini 6."
  },
  {
    "question": "Which product can split a 1/4\" TRS stereo output into two mono TS outputs?",
    "chunk_ids": [
      "B0B96LV4C5"
    ],
    "answer_example": "The HOSONGIN 1/4 TRS Stereo Jack to Dual 1/4 TS Mono Y-Splitter (B0B96LV4C5) splits a 1/4\" TRS stereo signal into left and right over two 1/4\" TS mono jacks.",
    "reasoning": "The chunk describes a TRS female to dual TS mono Y-splitter and its use-case in audio gear."
  },
  {
    "question": "What Wi-Fi range extender do you have that supports WiFi 6 and up to ~2100 sq. ft co

In [15]:
json_output = response.choices[0].message.content
json_output = json.loads(json_output)

In [16]:
json_output

[{'question': 'Do you have a stylus compatible with iPad Air 5th generation or iPad mini 6?',
  'chunk_ids': ['B0BF18F6R7'],
  'answer_example': 'Yes. The KEEPRO Pencil 2nd Generation (B0BF18F6R7) is listed as compatible with iPad Air (4th and 5th generation) and iPad mini (6th generation).',
  'reasoning': 'The product description explicitly lists compatible iPad models including iPad Air 5th gen and iPad mini 6.'},
 {'question': 'Which product can split a 1/4" TRS stereo output into two mono TS outputs?',
  'chunk_ids': ['B0B96LV4C5'],
  'answer_example': 'The HOSONGIN 1/4 TRS Stereo Jack to Dual 1/4 TS Mono Y-Splitter (B0B96LV4C5) splits a 1/4" TRS stereo signal into left and right over two 1/4" TS mono jacks.',
  'reasoning': 'The chunk describes a TRS female to dual TS mono Y-splitter and its use-case in audio gear.'},
 {'question': 'What Wi-Fi range extender do you have that supports WiFi 6 and up to ~2100 sq. ft coverage?',
  'chunk_ids': ['B0BZ5R7CVP'],
  'answer_example': 'Ten

In [17]:
len(json_output)

43

In [18]:
points = qdrant_client.scroll(
    collection_name="Amazon-items-collection-00",
    scroll_filter=Filter(
        must=[
            FieldCondition(
                key="parent_asin",
                match=MatchValue(value="B0BNVKS9WH")
            )
        ]
    ),
    limit=100,
    with_payload=True,
    with_vectors=False
)[0]

In [19]:
points[0].payload

{'description': 'HEYSONG Portable Bluetooth Speaker with Stereo Sound, 2000mAh Battery, IPX7 Waterproof Wireless Speakers for Outdoor, Home, Beach, Biking, Pool, TravelLoud Stereo Sound & Deep Bass: Featuring heysong unique audio processing through 2 drivers, it delivers surprisingly loud, no noticeable crackle at maximum volume. Complements your music without exaggeration or distortion. 100% Full Waterproof & Floating: Being IPX7 waterproof, this wireless Bluetooth speaker is protected against rain, spills, and splashes and can even survive submersion in up to 3 feet of water for up to 30 minutes. No worries about weather and outdoor condition, be the Rocker on your trip. Upgraded Stable Bluetooth 5.0: wireless Bluetooth speaker can normally connect the device from 66 unobstructed feets away even under difficult conditions and the signal is not disturbed. The Bluetooth 5.0 chip easily reads music playlists and connects to any device. 12 hours of playtime & Portable: This Bluetooth spe

In [20]:
def get_description(parent_asin: str) -> str:  
    
    points = qdrant_client.scroll(
        collection_name="Amazon-items-collection-00",
        scroll_filter=Filter(
            must=[
                FieldCondition(
                    key="parent_asin",
                    match=MatchValue(value=parent_asin)
                )
            ]
        ),
        limit=100,
        with_payload=True,
        with_vectors=False
    )[0]

    return points[0].payload["description"]

In [21]:
get_description("B0BNVKS9WH")

'HEYSONG Portable Bluetooth Speaker with Stereo Sound, 2000mAh Battery, IPX7 Waterproof Wireless Speakers for Outdoor, Home, Beach, Biking, Pool, TravelLoud Stereo Sound & Deep Bass: Featuring heysong unique audio processing through 2 drivers, it delivers surprisingly loud, no noticeable crackle at maximum volume. Complements your music without exaggeration or distortion. 100% Full Waterproof & Floating: Being IPX7 waterproof, this wireless Bluetooth speaker is protected against rain, spills, and splashes and can even survive submersion in up to 3 feet of water for up to 30 minutes. No worries about weather and outdoor condition, be the Rocker on your trip. Upgraded Stable Bluetooth 5.0: wireless Bluetooth speaker can normally connect the device from 66 unobstructed feets away even under difficult conditions and the signal is not disturbed. The Bluetooth 5.0 chip easily reads music playlists and connects to any device. 12 hours of playtime & Portable: This Bluetooth speaker has an unpa

### Create Eval dataset in Langsmith

In [33]:
client = Client(api_key=os.environ["LANGSMITH_API_KEY"])

In [34]:
dataset_name = "rag-evaluation-dataset"
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Dataset for evaluating RAG pipeline"
)

### Get list of all eval datasets

In [35]:
datasets = list(client.list_datasets())
print(f"Found {len(datasets)} dataset(s):")
for ds in datasets:
    print(f"  - Name: {ds.name}, ID: {ds.id}, Created: {ds.created_at}")

Found 1 dataset(s):
  - Name: rag-evaluation-dataset, ID: 030e9c26-20b4-42c0-843a-d7eccd876f7f, Created: 2026-01-18 09:58:42.045923+00:00


In [36]:
for item in json_output:
    client.create_example(
        dataset_id=dataset.id,
        inputs={"question": item["question"]},
        outputs={
            "ground_truth": item["answer_example"],
            "reference_context_ids": item["chunk_ids"],
            "reference_descriptions": [get_description(id) for id in item["chunk_ids"]]
        }
    )